In [1]:
import requests
import pandas as pd
import tqdm
import bs4
import json
import matplotlib.pyplot as plt
from adjustText import adjust_text
from pathlib import Path
import time

In [2]:
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2

from nba_api.stats.endpoints.playbyplay import PlayByPlay
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name
from nba_api.stats.endpoints.playergamelogs import PlayerGameLogs
from nba_api.stats.endpoints.boxscoretraditionalv2 import BoxScoreTraditionalV2
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail

In [3]:
path = Path('pbp_data/2020-21.csv')
data_frame = None
if path.exists():
    data_frame = pd.read_csv(str(path))

c:\users\danie\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data_frame.loc[:, 'GAME_ID'] = [f"00{item}" for item in data_frame.GAME_ID.astype(str)]

In [5]:
season = '2020-21'
print(f"Processing {season}")
pbp_dfs = []
game_finder = LeagueGameFinder(season_nullable=season, league_id_nullable='00', season_type_nullable='Regular Season')
df = game_finder.get_data_frames()[0]
game_ids = df.GAME_ID.unique()
for game_id in tqdm.tqdm(game_ids):
    if data_frame is not None and game_id in data_frame.GAME_ID.tolist():
        continue
    while True:
        try:
            pbp_df = PlayByPlayV2(game_id=game_id).get_data_frames()[0]
            break
        except Exception as e:
            print(f"Error occured: {e}, Game ID: {game_id}, retrying")
            time.sleep(0.1)
    pbp_dfs.append(pbp_df)
    time.sleep(0.25)
full_df = pd.concat([data_frame] + pbp_dfs)

Processing 2020-21


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1059/1059 [01:05<00:00, 16.11it/s]
c:\users\danie\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [6]:
full_df = pd.concat([data_frame] + pbp_dfs)

c:\users\danie\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [7]:
#full_df.loc[:, "PCTIMESECONDS"] = [720 - (int(item.split(":")[0]) * 60 + int(item.split(":")[1])) for item in full_df.PCTIMESTRING.tolist()]
full_df.loc[:, "PCTIMESECONDS"] = [720 - (int(item.PCTIMESTRING.split(":")[0]) * 60 + int(item.PCTIMESTRING.split(":")[1])) if item.PERIOD < 5 else 300 - (int(item.PCTIMESTRING.split(":")[0]) * 60 + int(item.PCTIMESTRING.split(":")[1])) for index, item in full_df.iterrows()]

full_df = full_df.fillna({
    "HOMEDESCRIPTION": "",
    "NEUTRALDESCRIPTION": "",
    "VISITORDESCRIPTION": ""
})

In [8]:
full_df.to_csv('pbp_data/2020-21.csv', index=False)


In [9]:
full_df = pd.read_csv('pbp_data/2020-21.csv')

In [10]:

full_df = full_df.fillna({
    "HOMEDESCRIPTION": "",
    "NEUTRALDESCRIPTION": "",
    "VISITORDESCRIPTION": ""
})
full_df.loc[:, 'GAME_ID'] = [f"00{item}" for item in full_df.GAME_ID.astype(str)]
#full_df.loc[:, "PCTIMESECONDS"] = [720 - (int(item.split(":")[0]) * 60 + int(item.split(":")[1])) for item in full_df.PCTIMESTRING.tolist()]
full_df.loc[:, "PCTIMESECONDS"] = [720 - (int(item.PCTIMESTRING.split(":")[0]) * 60 + int(item.PCTIMESTRING.split(":")[1])) if item.PERIOD < 5 else 300 - (int(item.PCTIMESTRING.split(":")[0]) * 60 + int(item.PCTIMESTRING.split(":")[1])) for index, item in full_df.iterrows()]

### Finder

In [ ]:
def get_game_schedule_for_season(season):
    game_finder = LeagueGameFinder(season_nullable=season, league_id_nullable='00', season_type_nullable='Regular Season')
    df = game_finder.get_data_frames()[0]
    return df

In [ ]:
def get_player_substitutions(pbp_df, player_id, game_schedule_df):
    """target_games = game_schedule_df.loc[game_schedule_df.TEAM_ID == team_id]
    team_games = pbp_df.loc[pbp_df.GAME_ID.isin(target_games.GAME_ID)]
    team_subs = team_games.loc[(
        (team_games.EVENTMSGTYPE == 8) & (team_games.PLAYER1_TEAM_ID == team_id)) 
    | (team_games.EVENTMSGTYPE == 12) 
    | (team_games.EVENTMSGTYPE == 13)]
    """
    player_subs = pbp_df.loc[(
        ( (pbp_df.EVENTMSGTYPE == 8) & ((pbp_df.PLAYER1_ID == player_id) | (pbp_df.PLAYER2_ID == player_id))) |
        ( pbp_df.EVENTMSGTYPE == 13) |
        ( pbp_df.EVENTMSGTYPE == 12))]
    return player_subs

In [ ]:
SUB_OUT = "SUB_OUT"
SUB_IN = "SUB_IN"
PERIOD_END = "PERIOD_END"

def filter_plays(
    df,
    previous_seconds: int,
    current_seconds: int,
    previous_eventnum: int,
    current_eventnum: int,
    prev_type: str,
    cur_type: str,
    target_player_id,
    stint_number: int,
    cur_period: int
):
    if cur_type == SUB_IN:
        return None
    
    """if previous_eventnum is None:
        indexed_df = df.loc[df.EVENTNUM <= current_eventnum]
    else:
        indexed_df = df.loc[(df.EVENTNUM > previous_eventnum) & (df.EVENTNUM <= current_eventnum)]"""
    indexed_df = df.loc[df.PERIOD == cur_period]
    if previous_seconds is None:
        indexed_df = indexed_df.loc[(indexed_df.PCTIMESECONDS <= current_seconds)]
    else:
        indexed_df = indexed_df.loc[(indexed_df.PCTIMESECONDS <= current_seconds) & (indexed_df.PCTIMESECONDS > previous_seconds)]
    subset_without_subs = indexed_df.loc[indexed_df.EVENTMSGTYPE != 8]
    if any(subset_without_subs.PLAYER1_ID == target_player_id) or \
        any(subset_without_subs.PLAYER2_ID == target_player_id) or \
        any(subset_without_subs.PLAYER3_ID == target_player_id):
        indexed_df.loc[:, 'STINT_NUM'] = stint_number
        return indexed_df
    return None

def filter_off_court(
    df,
    previous_seconds: int,
    current_seconds: int,
    previous_eventnum: int,
    current_eventnum: int,
    prev_type: str,
    cur_type: str,
    target_player_id,
    cur_period: int
):
    if cur_type == SUB_OUT:
        return None
    
    """if previous_eventnum is None:
        indexed_df = df.loc[df.EVENTNUM < current_eventnum]
    else:
        indexed_df = df.loc[(df.EVENTNUM > previous_eventnum) & (df.EVENTNUM < current_eventnum)]"""
    indexed_df = df.loc[df.PERIOD == cur_period]
    if previous_seconds is None:
        indexed_df = indexed_df.loc[(indexed_df.PCTIMESECONDS <= current_seconds)]
    else:
        indexed_df = indexed_df.loc[(indexed_df.PCTIMESECONDS <= current_seconds) & (indexed_df.PCTIMESECONDS > previous_seconds)]
    subset_without_subs = indexed_df.loc[indexed_df.EVENTMSGTYPE != 8]
    if all(subset_without_subs.PLAYER1_ID != target_player_id) and \
        all(subset_without_subs.PLAYER2_ID != target_player_id) and \
        all(subset_without_subs.PLAYER3_ID != target_player_id):
        return indexed_df
    return None

In [ ]:

def get_on_off_data(player_id, season, player_subs, full_df):
    final_dfs = []
    final_off_dfs = []
    pgl = PlayerGameLogs(player_id_nullable=player_id, season_nullable=season, measure_type_player_game_logs_nullable='Base')
    pgl = pgl.get_data_frames()[0]
    stint_number = 0
    for game_id in tqdm.tqdm(pgl.GAME_ID.unique()):
        cur_subs = player_subs.loc[player_subs.GAME_ID == game_id]
        key_events = cur_subs.loc[:, ["GAME_ID", "EVENTMSGTYPE", "EVENTNUM", "PERIOD", "PCTIMESTRING", "PLAYER1_ID", "PLAYER2_ID", "PCTIMESECONDS"]]

        current_game = full_df.loc[full_df.GAME_ID == game_id]
        previous_eventnum = None
        previous_seconds = 0
        prev_type = None
        target_dfs = []
        target_off_dfs = []
        for i in range(len(key_events)):
            current_event = key_events.iloc[i]
            ev_type = PERIOD_END
            if current_event.EVENTMSGTYPE == 8 and current_event.PLAYER1_ID == player_id:
                ev_type = SUB_OUT
            elif current_event.EVENTMSGTYPE == 8 and current_event.PLAYER2_ID == player_id:
                ev_type = SUB_IN
            elif current_event.EVENTMSGTYPE == 12:
                previous_eventnum = current_event.EVENTNUM
                previous_seconds = current_event.PCTIMESECONDS
                prev_type = ev_type
                continue
            filtered_df = filter_plays(
                df=current_game, 
                previous_seconds=previous_seconds,
                previous_eventnum=previous_eventnum,
                current_seconds=current_event.PCTIMESECONDS,
                current_eventnum=current_event.EVENTNUM,
                prev_type=prev_type,
                cur_type=ev_type,
                target_player_id=player_id,
                stint_number=stint_number,
                cur_period=current_event.PERIOD
            )
            if filtered_df is not None:
                stint_number += 1
                target_dfs.append(filtered_df)

            off_court_filter = filter_off_court(
                df=current_game, 
                previous_seconds=previous_seconds,
                previous_eventnum=previous_eventnum,
                current_seconds=current_event.PCTIMESECONDS,
                current_eventnum=current_event.EVENTNUM,
                cur_period=current_event.PERIOD,
                prev_type=prev_type,
                cur_type=ev_type,
                target_player_id=player_id
            )
            if off_court_filter is not None:
                target_off_dfs.append(off_court_filter)
            previous_eventnum = current_event.EVENTNUM
            previous_seconds = current_event.PCTIMESECONDS
            prev_type = ev_type

        if len(target_dfs) == 0:
            continue
        final_dfs.append(pd.concat(target_dfs))
        if len(target_off_dfs) == 0:
            continue
        final_off_dfs.append(pd.concat(target_off_dfs))
    player_on = pd.concat(final_dfs)
    player_off = pd.concat(final_off_dfs)
    return player_on, player_off

In [ ]:
from collections import Counter


def get_shots(player_on):
    made_shots = Counter()
    all_shots = Counter()

    stints = list(player_on.STINT_NUM.unique())
    for stint in stints:
        cur_stint = player_on.loc[player_on.STINT_NUM == stint]
        stint_start = cur_stint.PCTIMESECONDS.min()
        stint_end = cur_stint.PCTIMESECONDS.max()
        cur_shots = cur_stint.loc[((cur_stint.EVENTMSGTYPE == 1) | (cur_stint.EVENTMSGTYPE == 2)) & (cur_stint.PLAYER1_ID == player_id)]
        cur_shots = cur_shots.drop_duplicates()
        for i in range(len(cur_shots)):
            cur_shot = cur_shots.iloc[i]
            time_of_shot = cur_shot.PCTIMESECONDS - stint_start
            # Shot was taken in a stint with < bucket
            bucket = (time_of_shot // 60) + 1
            if cur_shot.EVENTMSGTYPE == 1:
                made_shots[bucket] += 1

            all_shots[bucket] += 1
    return made_shots, all_shots

In [ ]:
def print_efficiency(made_shots, all_shots, player_name):
    #for key in sorted(all_shots):
    #    print(f"Efficiency when {player_name} is {key-1}-{key} minute on court: {made_shots[key]}/{all_shots[key]} ({made_shots[key]/all_shots[key] * 100:.1f}%)")

    #print()
    first_quartal = []

    ret_val = []
    for key in [3, 6, 9, 12]:
        fgm = 0
        fga = 0
        for item in range(key-3+1, key+1):
            fgm += made_shots[item]
            fga += all_shots[item]
        ret_val.append(f"Efficiency when {player_name} is {key-3}-{key} minutes on court: {fgm}/{fga} ({fgm/fga * 100:.1f}%)")
    return "\n".join(ret_val)

## Checking for Jokic

In [ ]:
player_name = "Nikola Jokic"
team_name = "Denver Nuggets"
season = '2020-21'
player_id = find_players_by_full_name(player_name)[0]['id']
team_id = find_teams_by_full_name(team_name)[0]['id']

In [ ]:
last_schedule = get_game_schedule_for_season(season=season)

player_subs = get_player_substitutions(pbp_df=full_df, player_id=player_id, game_schedule_df=last_schedule)

player_on, player_off = get_on_off_data(player_id=player_id, season=season, player_subs=player_subs, full_df=full_df)

In [ ]:
single_game.loc[single_game.PERIOD == 3]

In [ ]:
box_score = BoxScoreTraditionalV2(game_id='0022000267', start_range='14410', end_range=f'{str(14400+240)}', range_type='2')

In [ ]:
box_score.get_data_frames()[0]

In [ ]:
team_1_players = []

team_2_players = []

teams_playing = game_finder_df.loc[game_finder_df.GAME_ID == single_game.GAME_ID.iloc[0]]

team_1_id = teams_playing.iloc[0].TEAM_ID
team_2_id = teams_playing.iloc[1].TEAM_ID

for index in range(len(single_game)):
    item = single_game.iloc[index]
    if (item.EVENTMSGTYPE == 12):
        start_period = ((item.PERIOD-1) * 12 ) * 600 + 10
        end_period = ((item.PERIOD-1) * 12 ) * 600 + 240
        box_score = BoxScoreTraditionalV2(game_id=item.GAME_ID, start_range=str(start_period), end_range=str(end_period), range_type='2')
        players_on = box_score.get_data_frames()[0]
        players_on.loc[:, "SECONDS"] = [(int(item.split(":")[0]) * 60 + int(item.split(":")[1])) for item in players_on.MIN.tolist()]
        team_1_on = players_on.loc[players_on.TEAM_ID == team_1_id].sort_values(by='SECONDS', ascending=False).iloc[:5]
        team_2_on = players_on.loc[players_on.TEAM_ID == team_2_id].sort_values(by='SECONDS', ascending=False).iloc[:5]        
        team_1_players.append(team_1_on.PLAYER_ID.tolist())
        team_2_players.append(team_2_on.PLAYER_ID.tolist())
        continue
    cur_team_1 = team_1_players[-1][:]
    cur_team_2 = team_2_players[-1][:]
    if (item.EVENTMSGTYPE == 8):
        player_in = item.PLAYER2_ID
        player_out = item.PLAYER1_ID
        is_team_1 = item.PLAYER1_TEAM_ID == team_1_id
        if is_team_1:
            cur_team_1[cur_team_1.index(player_out)] = player_in
        else:
            cur_team_2[cur_team_2.index(player_out)] = player_in
    team_1_players.append(cur_team_1)
    team_2_players.append(cur_team_2)

## CALCULATING WHICH PLAYERS ARE ON FLOOR AT THE TIME

In [9]:
def find_starters_in_period(found_players, period_pbp):
    #period_start = single_game.loc[single_game.PERIOD == 2] -> period_pbp
    num_subs = len(found_players) - 5
    sub_count = 0
    subbed_in = []
    for i in range(len(period_pbp)):
        item = period_pbp.iloc[i]
        if item.EVENTMSGTYPE == 8:
            player_in = item.PLAYER2_ID
            subbed_in.append(player_in)
            sub_count += 1
        if sub_count == num_subs:
            break

    return found_players.loc[~found_players.PLAYER_ID.isin(subbed_in)].PLAYER_ID.tolist()


def do_work(team_1_players, team_2_players, team_1_id, team_2_id, single_game, item):
    #item = single_game.iloc[index]
    if (item.EVENTMSGTYPE == 12):
        period = item.PERIOD
        if period > 4:
            start_period = ((4 * 12) + (period-5) * 5) * 600 + 0
            end_period = ((4 * 12) + (period-5) * 5) * 600 + 480
        else:
            start_period = ((period-1) * 12 ) * 600 + 15
            end_period = ((period-1) * 12 ) * 600 + 480

        # Fetch players at the start of quarter
        box_score = BoxScoreTraditionalV2(game_id=item.GAME_ID, start_range=str(start_period), end_range=str(end_period), range_type='2')
        players_on = box_score.get_data_frames()[0]
        players_on.loc[:, "SECONDS"] = [(int(item.split(":")[0]) * 60 + int(item.split(":")[1])) for item in players_on.MIN.tolist()]

        # Split by team
        team_1_on = players_on.loc[players_on.TEAM_ID == team_1_id]
        team_2_on = players_on.loc[players_on.TEAM_ID == team_2_id]
        period_pbp = single_game.loc[single_game.PERIOD == item.PERIOD]

        # Check if more than 5 players are found
        if len(team_1_on) > 5:
            team_1_players.append(find_starters_in_period(team_1_on, period_pbp))
        else:
            team_1_players.append(team_1_on.PLAYER_ID.tolist())
        if len(team_1_on) > 5:
            team_2_players.append(find_starters_in_period(team_2_on, period_pbp))
        else:
            team_2_players.append(team_2_on.PLAYER_ID.tolist())
        time.sleep(0.25)
        return
    cur_team_1 = team_1_players[-1][:]
    cur_team_2 = team_2_players[-1][:]
    if (item.EVENTMSGTYPE == 8):
        player_in = item.PLAYER2_ID
        player_out = item.PLAYER1_ID
        is_team_1 = item.PLAYER1_TEAM_ID == team_1_id
        if is_team_1:
            if player_out in cur_team_1:
                cur_team_1[cur_team_1.index(player_out)] = player_in
        else:
            if player_out in cur_team_2:
                cur_team_2[cur_team_2.index(player_out)] = player_in
    team_1_players.append(cur_team_1)
    team_2_players.append(cur_team_2)


In [10]:
cur_pbp_players = pd.read_csv('pbp_players/2020-21.csv')
cur_pbp_players.loc[:, 'GAME_ID'] = [f"00{item}" for item in cur_pbp_players.GAME_ID.astype(str)]

In [11]:
game_finder = LeagueGameFinder(season_nullable='2020-21', league_id_nullable='00', season_type_nullable='Regular Season')
game_finder_df = game_finder.get_data_frames()[0]
modified_dfs = []
#start = time.time()
count = 0
for index, game_id in enumerate(full_df.GAME_ID.unique()):
    print(f"{index}/{len(full_df.GAME_ID.unique())}: {game_id}")
    if game_id in cur_pbp_players.GAME_ID.tolist():
        continue
    #if game_id in formatted_df.GAME_ID.unique():
    #continue
    single_game = full_df.loc[full_df.GAME_ID == game_id]
    team_1_players = []

    team_2_players = []

    teams_playing = game_finder_df.loc[game_finder_df.GAME_ID == single_game.GAME_ID.iloc[0]]

    team_1_id = teams_playing.iloc[0].TEAM_ID
    team_2_id = teams_playing.iloc[1].TEAM_ID

    #for index in range(len(single_game)):
    """
    for index in range(len(single_game)):
        item = single_game.iloc[index]
        do_work(team_1_players=team_1_players, team_2_players=team_2_players, team_1_id=team_1_id, team_2_id=team_2_id, single_game=single_game, item=item)
    """
    
    #"""
    for index, item in single_game.iterrows():
        do_work(team_1_players=team_1_players, team_2_players=team_2_players, team_1_id=team_1_id, team_2_id=team_2_id, single_game=single_game, item=item)
    #"""
    
    #single_game.apply(lambda item: do_work(team_1_players=team_1_players, team_2_players=team_2_players, team_1_id=team_1_id, team_2_id=team_2_id, single_game=single_game, item=item), axis=1)
    
    for i in range(len(team_1_players[0])):
        #team_1_dict[f'TEAM_1_PLAYER_{i+1}'] = [item[i] for item in team_1_players]
        single_game.loc[:, f'TEAM_1_PLAYER_{i+1}'] = [item[i] for item in team_1_players]
        
    for i in range(len(team_1_players[0])):
        #team_2_dict[f'TEAM_2_PLAYER_{i+1}'] = [item[i] for item in team_2_players]
        single_game.loc[:, f'TEAM_2_PLAYER_{i+1}'] = [item[i] for item in team_2_players]
        
    modified_dfs.append(single_game)
    count += 1
#end = time.time()
#print(f"End-Start: {end-start}")

0/1059: 0022000267
1/1059: 0022000262
2/1059: 0022000260
3/1059: 0022000261
4/1059: 0022000265
5/1059: 0022000266
6/1059: 0022000258
7/1059: 0022000257
8/1059: 0022000259
9/1059: 0022000251
10/1059: 0022000256
11/1059: 0022000254
12/1059: 0022000249
13/1059: 0022000252
14/1059: 0022000253
15/1059: 0022000250
16/1059: 0022000243
17/1059: 0022000242
18/1059: 0022000246
19/1059: 0022000245
20/1059: 0022000244
21/1059: 0022000247
22/1059: 0022000248
23/1059: 0022000230
24/1059: 0022000233
25/1059: 0022000232
26/1059: 0022000231
27/1059: 0022000239
28/1059: 0022000234
29/1059: 0022000229
30/1059: 0022000237
31/1059: 0022000241
32/1059: 0022000236
33/1059: 0022000238
34/1059: 0022000227
35/1059: 0022000228
36/1059: 0022000226
37/1059: 0022000220
38/1059: 0022000217
39/1059: 0022000218
40/1059: 0022000216
41/1059: 0022000223
42/1059: 0022000224
43/1059: 0022000219
44/1059: 0022000221
45/1059: 0022000222
46/1059: 0022000214
47/1059: 0022000213
48/1059: 0022000211
49/1059: 0022000208
50/1059: 0

785/1059: 0022000831
786/1059: 0022000461
787/1059: 0022000829
788/1059: 0022000817
789/1059: 0022000826
790/1059: 0022000828
791/1059: 0022000827
792/1059: 0022000825
793/1059: 0022000823
794/1059: 0022000824
795/1059: 0022000815
796/1059: 0022000278
797/1059: 0022000819
798/1059: 0022000818
799/1059: 0022000816
800/1059: 0022000822
801/1059: 0022000820
802/1059: 0022000821
803/1059: 0022000812
804/1059: 0022000806
805/1059: 0022000813
806/1059: 0022000811
807/1059: 0022000809
808/1059: 0022000805
809/1059: 0022000807
810/1059: 0022000814
811/1059: 0022000810
812/1059: 0022000808
813/1059: 0022000802
814/1059: 0022000803
815/1059: 0022000804
816/1059: 0022000800
817/1059: 0022000801
818/1059: 0022000794
819/1059: 0022000199
820/1059: 0022000796
821/1059: 0022000790
822/1059: 0022000797
823/1059: 0022000793
824/1059: 0022000798
825/1059: 0022000795
826/1059: 0022000799
827/1059: 0022000792
828/1059: 0022000791
829/1059: 0022000784
830/1059: 0022000787
831/1059: 0022000523
832/1059: 002

c:\users\danie\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\danie\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


988/1059: 0022001057
989/1059: 0022001055
990/1059: 0022000263
991/1059: 0022001059
992/1059: 0022001056
993/1059: 0022001054
994/1059: 0022000204
995/1059: 0022001047
996/1059: 0022001051
997/1059: 0022001053
998/1059: 0022001052
999/1059: 0022001048
1000/1059: 0022001050
1001/1059: 0022000255
1002/1059: 0022000458
1003/1059: 0022001049
1004/1059: 0022000154
1005/1059: 0022001043
1006/1059: 0022001044
1007/1059: 0022001046
1008/1059: 0022001042
1009/1059: 0022001045
1010/1059: 0022001039
1011/1059: 0022001034
1012/1059: 0022001040
1013/1059: 0022001035
1014/1059: 0022001041
1015/1059: 0022001033
1016/1059: 0022001038
1017/1059: 0022001036
1018/1059: 0022001037
1019/1059: 0022001032
1020/1059: 0022000448
1021/1059: 0022001026
1022/1059: 0022001028
1023/1059: 0022001030
1024/1059: 0022001029
1025/1059: 0022001031
1026/1059: 0022001027
1027/1059: 0022001025
1028/1059: 0022001023
1029/1059: 0022001020
1030/1059: 0022001024
1031/1059: 0022001021
1032/1059: 0022000145
1033/1059: 0022001022


In [31]:
len(a)

250

In [17]:
lista

[[2546, 203496, 202683, 1626181, 203468],
 [2546, 203496, 202683, 1626181, 203468],
 [2546, 203496, 202683, 1626181, 203468],
 [2546, 203496, 202683, 1626181, 203468],
 [2546, 203496, 202683, 1626181, 203468],
 [2546, 203496, 202683, 1626181, 203468]]

In [14]:
temp = pd.concat(modified_dfs)

Normal: End-Start: 31.241218328475952

Iterrows: End-Start: 6.60307765007019

Apply: 6.267292022705078

In [15]:
len(modified_dfs)

71

In [16]:
len(cur_pbp_players.GAME_ID.unique())

859

In [31]:
533+122

655

In [35]:
formatted_df = pd.concat([cur_pbp_players, temp, pd.concat(modified_dfs)])

In [12]:
formatted_df = pd.concat([cur_pbp_players, pd.concat(modified_dfs)])

In [13]:
len(formatted_df.GAME_ID.unique())

1059

In [14]:
formatted_df.to_csv('pbp_players/2020-21.csv', index=False)

### Jokic analysis

In [340]:
jokic_id = find_players_by_full_name("Nikola Jokic")[0]['id']

In [37]:
gobert_id = find_players_by_full_name("Rudy Gobert")[0]['id']
jazz_id = find_teams_by_full_name("Utah Jazz")[0]['id']

In [33]:
target_player_columns = [
    'TEAM_1_PLAYER_1', 'TEAM_1_PLAYER_2', 'TEAM_1_PLAYER_3',
    'TEAM_1_PLAYER_4', 'TEAM_1_PLAYER_5', 'TEAM_2_PLAYER_1',
    'TEAM_2_PLAYER_2', 'TEAM_2_PLAYER_3', 'TEAM_2_PLAYER_4',
    'TEAM_2_PLAYER_5'
]

def get_on_floor(player_id, df):
    queries = [df[key] == player_id for key in target_player_columns]
    query = queries[0]
    for cur in queries[1:]:
        query = query | cur
    return df.loc[query]


def get_off_floor(player_id, df):
    queries = [df[key] != player_id for key in target_player_columns]
    query = queries[0]
    for cur in queries[1:]:
        query = query & cur
    return df.loc[query]


def check_any(item, player_id):
    for col in target_player_columns:
        if item[col] == player_id:
            return True
    return False


In [69]:
player_on_floor = get_on_floor(gobert_id, formatted_df)

In [70]:
player_off_floor = get_off_floor(gobert_id, formatted_df)

In [38]:
shotchart_df = ShotChartDetail(player_id=0, team_id=jazz_id, context_measure_simple='FGA', season_nullable='2020-21')

In [39]:
shotchart_df = shotchart_df.get_data_frames()[0]

In [128]:
shotchart_all = ShotChartDetail(player_id=0, team_id=0, context_measure_simple='FGA', season_nullable='2020-21').get_data_frames()[0]

In [72]:
shot_off_court = pd.merge(left=shotchart_df, right=player_off_floor, left_on=('GAME_ID', 'GAME_EVENT_ID'), right_on=('GAME_ID', 'EVENTNUM'))
shot_on_court = pd.merge(left=shotchart_df, right=player_on_floor, left_on=('GAME_ID', 'GAME_EVENT_ID'), right_on=('GAME_ID', 'EVENTNUM'))

In [99]:
grouped_off = shot_off_court.groupby(['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_ZONE_BASIC']).sum().reset_index()
grouped_off = grouped_off.loc[:, ['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_ZONE_BASIC', 'SHOT_MADE_FLAG', 'SHOT_ATTEMPTED_FLAG']]
grouped_off.loc[:, 'FG_PCT'] = grouped_off.SHOT_MADE_FLAG / grouped_on.SHOT_ATTEMPTED_FLAG * 100
grouped_off.loc[:, 'PCT_OF_SHOTS'] = grouped_off.SHOT_ATTEMPTED_FLAG / grouped_on.SHOT_ATTEMPTED_FLAG.sum() * 100

In [100]:
grouped_on = shot_on_court.groupby(['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_ZONE_BASIC']).sum().reset_index()
grouped_on = grouped_on.loc[:, ['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_ZONE_BASIC', 'SHOT_MADE_FLAG', 'SHOT_ATTEMPTED_FLAG']]
grouped_on.loc[:, 'FG_PCT'] = grouped_on.SHOT_MADE_FLAG / grouped_on.SHOT_ATTEMPTED_FLAG * 100
grouped_on.loc[:, 'PCT_OF_SHOTS'] = grouped_on.SHOT_ATTEMPTED_FLAG / grouped_on.SHOT_ATTEMPTED_FLAG.sum() * 100

In [159]:
player_names = [
    "Donovan Mitchell",
    "Bojan Bogdanovic",
    "Mike Conley",
    "Jordan Clarkson",
    "Royce O'Neale",
    "Rudy Gobert",
    "Joe Ingles"
]

In [160]:
data = {}

for player_name in player_names:
    pl_id = shotchart_df.loc[shotchart_df.PLAYER_NAME == player_name].iloc[0].PLAYER_ID
    
    player_on_floor = get_on_floor(pl_id, formatted_df)
    player_off_floor = get_off_floor(pl_id, formatted_df)
    
    shot_off_court = pd.merge(left=shotchart_df, right=player_off_floor, left_on=('GAME_ID', 'GAME_EVENT_ID'), right_on=('GAME_ID', 'EVENTNUM'))
    shot_on_court = pd.merge(left=shotchart_df, right=player_on_floor, left_on=('GAME_ID', 'GAME_EVENT_ID'), right_on=('GAME_ID', 'EVENTNUM'))
    
    grouped_off = shot_off_court.groupby(['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_ZONE_BASIC']).sum().reset_index()
    grouped_off = grouped_off.loc[:, ['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_ZONE_BASIC', 'SHOT_MADE_FLAG', 'SHOT_ATTEMPTED_FLAG']]
    grouped_off.loc[:, 'FG_PCT'] = grouped_off.SHOT_MADE_FLAG / grouped_off.SHOT_ATTEMPTED_FLAG * 100
    grouped_off.loc[:, 'PCT_OF_SHOTS'] = grouped_off.SHOT_ATTEMPTED_FLAG / grouped_off.SHOT_ATTEMPTED_FLAG.sum() * 100

    grouped_on = shot_on_court.groupby(['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_ZONE_BASIC']).sum().reset_index()
    grouped_on = grouped_on.loc[:, ['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_ZONE_BASIC', 'SHOT_MADE_FLAG', 'SHOT_ATTEMPTED_FLAG']]
    grouped_on.loc[:, 'FG_PCT'] = grouped_on.SHOT_MADE_FLAG / grouped_on.SHOT_ATTEMPTED_FLAG * 100
    grouped_on.loc[:, 'PCT_OF_SHOTS'] = grouped_on.SHOT_ATTEMPTED_FLAG / grouped_on.SHOT_ATTEMPTED_FLAG.sum() * 100
    
    #grouped_on.loc[grouped_on.SHOT_ZONE_BASIC == 'Left Corner 3']
    
    corner_total_made_on = grouped_on.loc[grouped_on.SHOT_ZONE_BASIC == 'Left Corner 3'].iloc[0].SHOT_MADE_FLAG + grouped_on.loc[grouped_on.SHOT_ZONE_BASIC == 'Right Corner 3'].iloc[0].SHOT_MADE_FLAG
    corner_total_on = grouped_on.loc[grouped_on.SHOT_ZONE_BASIC == 'Left Corner 3'].iloc[0].SHOT_ATTEMPTED_FLAG + grouped_on.loc[grouped_on.SHOT_ZONE_BASIC == 'Right Corner 3'].iloc[0].SHOT_ATTEMPTED_FLAG

    corner_total_made_off = grouped_off.loc[grouped_off.SHOT_ZONE_BASIC == 'Left Corner 3'].iloc[0].SHOT_MADE_FLAG + grouped_off.loc[grouped_off.SHOT_ZONE_BASIC == 'Right Corner 3'].iloc[0].SHOT_MADE_FLAG
    corner_total_off = grouped_off.loc[grouped_off.SHOT_ZONE_BASIC == 'Left Corner 3'].iloc[0].SHOT_ATTEMPTED_FLAG + grouped_off.loc[grouped_off.SHOT_ZONE_BASIC == 'Right Corner 3'].iloc[0].SHOT_ATTEMPTED_FLAG
    
    
    data[player_name] = {
        'ON_LEFT_CORNER_FG_PCT': grouped_on.loc[grouped_on.SHOT_ZONE_BASIC == 'Left Corner 3'].iloc[0].FG_PCT,
        'ON_LEFT_CORNER_PCT_SHOTS': grouped_on.loc[grouped_on.SHOT_ZONE_BASIC == 'Left Corner 3'].iloc[0].PCT_OF_SHOTS,
        'ON_RIGHT_CORNER_FG_PCT': grouped_on.loc[grouped_on.SHOT_ZONE_BASIC == 'Right Corner 3'].iloc[0].FG_PCT,
        'ON_RIGHT_CORNER_PCT_SHOTS': grouped_on.loc[grouped_on.SHOT_ZONE_BASIC == 'Right Corner 3'].iloc[0].PCT_OF_SHOTS,
        'OFF_LEFT_CORNER_FG_PCT': grouped_off.loc[grouped_off.SHOT_ZONE_BASIC == 'Left Corner 3'].iloc[0].FG_PCT,
        'OFF_LEFT_CORNER_PCT_SHOTS': grouped_off.loc[grouped_off.SHOT_ZONE_BASIC == 'Left Corner 3'].iloc[0].PCT_OF_SHOTS,
        'OFF_RIGHT_CORNER_FG_PCT': grouped_off.loc[grouped_off.SHOT_ZONE_BASIC == 'Right Corner 3'].iloc[0].FG_PCT,
        'OFF_RIGHT_CORNER_PCT_SHOTS': grouped_off.loc[grouped_off.SHOT_ZONE_BASIC == 'Right Corner 3'].iloc[0].PCT_OF_SHOTS,
        'ON_TOTAL_CORNER_FGM': corner_total_made_on,
        'ON_TOTAL_CORNER_FGA': corner_total_on,
        'OFF_TOTAL_CORNER_FGM': corner_total_made_off,
        'OFF_TOTAL_CORNER_FGA': corner_total_off,
    }

In [127]:
for player_name in data:
    print(player_name)
    #for key in data[player_name]:
        #print(f"\t{key}: {data[player_name][key]}")
    total_sum_on = data[player_name]['ON_LEFT_CORNER_PCT_SHOTS'] + data[player_name]['ON_RIGHT_CORNER_PCT_SHOTS']
    total_sum_off = data[player_name]['OFF_LEFT_CORNER_PCT_SHOTS'] + data[player_name]['OFF_RIGHT_CORNER_PCT_SHOTS']
    print(f"\tTotal Diff in corner ratio on-off: {total_sum_on-total_sum_off}")

Donovan Mitchell
	Total Diff in corner ratio on-off: 1.7166078006911984
Bojan Bogdanovic
	Total Diff in corner ratio on-off: 1.422449182132322
Mike Conley
	Total Diff in corner ratio on-off: -1.434219595087816
Jordan Clarkson
	Total Diff in corner ratio on-off: -0.622037020261498
Royce O'Neale
	Total Diff in corner ratio on-off: -0.5471956224350194
Rudy Gobert
	Total Diff in corner ratio on-off: 0.1928083640107694
Joe Ingles
	Total Diff in corner ratio on-off: 0.6701510727774345


#### Gobert on-off defensively

In [147]:
pl_id = gobert_id
    
player_on_floor = get_on_floor(pl_id, formatted_df)
player_off_floor = get_off_floor(pl_id, formatted_df)
player_off_floor = player_off_floor.loc[player_off_floor.GAME_ID.isin(player_on_floor.GAME_ID.unique())]

shot_off_court = pd.merge(left=shotchart_all, right=player_off_floor, left_on=('GAME_ID', 'GAME_EVENT_ID'), right_on=('GAME_ID', 'EVENTNUM'))
shot_on_court = pd.merge(left=shotchart_all, right=player_on_floor, left_on=('GAME_ID', 'GAME_EVENT_ID'), right_on=('GAME_ID', 'EVENTNUM'))

In [148]:
non_utah_on_court = shot_on_court.loc[shot_on_court.TEAM_ID != jazz_id]
non_utah_off_court = shot_off_court.loc[shot_off_court.TEAM_ID != jazz_id]

In [152]:
#key = ['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_ZONE_BASIC']
key = ['SHOT_ZONE_BASIC']

In [153]:
grouped_off = non_utah_off_court.groupby(key).sum().reset_index()
grouped_off = grouped_off.loc[:, key + ['SHOT_MADE_FLAG', 'SHOT_ATTEMPTED_FLAG']]
grouped_off.loc[:, 'FG_PCT'] = grouped_off.SHOT_MADE_FLAG / grouped_off.SHOT_ATTEMPTED_FLAG * 100
grouped_off.loc[:, 'PCT_OF_SHOTS'] = grouped_off.SHOT_ATTEMPTED_FLAG / grouped_off.SHOT_ATTEMPTED_FLAG.sum() * 100

grouped_on = non_utah_on_court.groupby(key).sum().reset_index()
grouped_on = grouped_on.loc[:, key + ['SHOT_MADE_FLAG', 'SHOT_ATTEMPTED_FLAG']]
grouped_on.loc[:, 'FG_PCT'] = grouped_on.SHOT_MADE_FLAG / grouped_on.SHOT_ATTEMPTED_FLAG * 100
grouped_on.loc[:, 'PCT_OF_SHOTS'] = grouped_on.SHOT_ATTEMPTED_FLAG / grouped_on.SHOT_ATTEMPTED_FLAG.sum() * 100

In [155]:
merged_gobert_def = pd.merge(left=grouped_on, right=grouped_off, on=key, suffixes=('_ON', '_OFF'))

In [158]:
merged_gobert_def

,SHOT_ZONE_BASIC,SHOT_MADE_FLAG_ON,SHOT_ATTEMPTED_FLAG_ON,FG_PCT_ON,PCT_OF_SHOTS_ON,SHOT_MADE_FLAG_OFF,SHOT_ATTEMPTED_FLAG_OFF,FG_PCT_OFF,PCT_OF_SHOTS_OFF
0,Above the Break 3,133,403,33.002481,27.716644,82,236,34.745763,28.095238
1,In The Paint (Non-RA),111,308,36.038961,21.182944,71,175,40.571429,20.833333
2,Left Corner 3,20,51,39.215686,3.507565,10,23,43.478261,2.738095
3,Mid-Range,95,239,39.748954,16.437414,34,102,33.333333,12.142857
4,Restricted Area,236,403,58.560794,27.716644,182,281,64.768683,33.452381
5,Right Corner 3,24,49,48.979592,3.370014,8,23,34.782609,2.738095
